# ReAct Pattern with OpenAI GPT-4o

ReAct (Reasoning + Acting) 패턴은 LLM이 **추론(Reasoning)**과 **행동(Acting)**을 번갈아가며 수행하는 에이전트 패턴입니다.

## 동작 방식
1. **Thought**: 현재 상황을 분석하고 다음 행동을 결정
2. **Action**: 도구(Tool)를 실행
3. **Observation**: 도구 실행 결과를 관찰
4. 목표를 달성할 때까지 1-3 반복

이 노트북에서는 **LangGraph**를 사용하여 ReAct 에이전트를 구현합니다.

## 1. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

print("환경 설정 완료!")

## 2. 도구(Tools) 정의

ReAct 에이전트가 사용할 도구들을 정의합니다.

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """두 숫자를 더합니다."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """두 숫자를 곱합니다."""
    return a * b


@tool
def divide(a: int, b: int) -> float:
    """첫 번째 숫자를 두 번째 숫자로 나눕니다."""
    if b == 0:
        return "Error: 0으로 나눌 수 없습니다."
    return a / b


@tool
def get_weather(city: str) -> str:
    """도시의 현재 날씨 정보를 가져옵니다. (시뮬레이션)"""
    # 실제로는 API를 호출하지만, 여기서는 시뮬레이션
    weather_data = {
        "서울": "맑음, 15°C",
        "부산": "흐림, 18°C",
        "제주": "비, 20°C",
        "대전": "맑음, 16°C",
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다.")


# 사용할 도구 목록
tools = [add, multiply, divide, get_weather]
print(f"정의된 도구: {[t.name for t in tools]}")

## 3. LLM 설정 (OpenAI GPT-4o)

In [ ]:
from langchain_openai import ChatOpenAI

# GPT-4o 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,  # 일관된 출력을 위해 0으로 설정
)

# 도구를 바인딩한 LLM
llm_with_tools = llm.bind_tools(tools)

print("GPT-4o 모델 설정 완료!")

## 4. LangGraph를 사용한 ReAct 에이전트 구현

### 4.1 방법 1: `create_react_agent` 사용 (간단한 방법)

> **Note**: LangGraph v1.0에서 deprecated 경고가 나오지만 정상 작동합니다. v2.0에서 제거될 예정입니다.

In [ ]:
from langgraph.prebuilt import create_react_agent

# ReAct 에이전트 생성 (deprecated 경고가 나오지만 정상 작동)
react_agent = create_react_agent(llm, tools)

print("ReAct 에이전트 생성 완료!")

In [ ]:
# 간단한 테스트
response = react_agent.invoke({"messages": [("human", "3과 5를 더한 다음, 그 결과에 2를 곱해줘")]})

# 결과 출력
for message in response["messages"]:
    print(f"[{message.type}]: {message.content}")
    if hasattr(message, "tool_calls") and message.tool_calls:
        print(f"  -> Tool calls: {message.tool_calls}")

### 4.2 방법 2: 수동으로 ReAct 그래프 구성 (상세 이해용)

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode


# 상태 정의
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


# 에이전트 노드: LLM을 호출하여 다음 행동 결정
def agent_node(state: AgentState) -> AgentState:
    """에이전트가 추론하고 다음 행동을 결정합니다."""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}


# 조건부 라우팅: 도구 호출 여부에 따라 분기
def should_continue(state: AgentState) -> str:
    """도구를 호출해야 하는지 결정합니다."""
    last_message = state["messages"][-1]
    
    # 도구 호출이 있으면 tools 노드로
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    # 없으면 종료
    return END


# 도구 노드 생성
tool_node = ToolNode(tools)

# 그래프 구성
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 엣지 추가
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")  # 도구 실행 후 다시 에이전트로

# 그래프 컴파일
custom_react_agent = workflow.compile()

print("수동 ReAct 그래프 구성 완료!")

In [ ]:
# 그래프 시각화 (선택사항)
try:
    from IPython.display import Image, display
    display(Image(custom_react_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"그래프 시각화 실패: {e}")
    print("그래프 구조: START -> agent -> (tools -> agent) | END")

## 5. 다양한 테스트 케이스

In [ ]:
def run_agent(query: str, agent=react_agent):
    """에이전트를 실행하고 결과를 출력합니다."""
    print(f"\n{'='*60}")
    print(f"질문: {query}")
    print(f"{'='*60}")
    
    response = agent.invoke({"messages": [("human", query)]})
    
    for msg in response["messages"]:
        if msg.type == "human":
            continue
        elif msg.type == "ai":
            if hasattr(msg, "tool_calls") and msg.tool_calls:
                print(f"\n[AI - 도구 호출]")
                for tc in msg.tool_calls:
                    print(f"  -> {tc['name']}({tc['args']})")
            elif msg.content:
                print(f"\n[AI - 최종 응답]")
                print(f"  {msg.content}")
        elif msg.type == "tool":
            print(f"\n[도구 실행 결과] {msg.name}: {msg.content}")
    
    return response

In [ ]:
# 테스트 1: 수학 연산
run_agent("10과 20을 더하고, 그 결과를 3으로 나눠줘")

In [ ]:
# 테스트 2: 날씨 정보
run_agent("서울과 부산의 날씨를 알려줘")

In [ ]:
# 테스트 3: 복합 작업
run_agent("제주도 날씨를 확인하고, 온도가 20도면 5를 곱하고 아니면 3을 곱해줘")

## 6. 스트리밍 출력

에이전트의 추론 과정을 실시간으로 확인할 수 있습니다.

In [ ]:
def run_agent_stream(query: str, agent=react_agent):
    """에이전트를 스트리밍 모드로 실행합니다."""
    print(f"\n{'='*60}")
    print(f"질문: {query}")
    print(f"{'='*60}")
    
    for event in agent.stream({"messages": [("human", query)]}, stream_mode="updates"):
        for node_name, node_output in event.items():
            print(f"\n[{node_name} 노드 실행]")
            if "messages" in node_output:
                for msg in node_output["messages"]:
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"  -> 도구 호출: {tc['name']}({tc['args']})")
                    elif hasattr(msg, "content") and msg.content:
                        print(f"  -> {msg.content[:200]}..." if len(str(msg.content)) > 200 else f"  -> {msg.content}")

In [ ]:
# 스트리밍 테스트
run_agent_stream("5와 7을 곱한 다음, 결과에 10을 더해줘")

## 7. 요약

### ReAct 패턴의 핵심
- **Reasoning**: LLM이 현재 상황을 분석하고 계획을 세움
- **Acting**: 정의된 도구를 실행하여 정보를 수집하거나 작업 수행
- **Observation**: 도구 실행 결과를 관찰하고 다음 단계 결정

### LangGraph 사용의 장점
- 상태 관리가 명확함
- 그래프 구조로 복잡한 워크플로우 구성 가능
- 스트리밍 지원으로 실시간 모니터링 가능
- 체크포인트를 통한 상태 저장/복원 지원

### GPT-4o 모델 특징
- 뛰어난 추론 능력
- 도구 호출 정확도가 높음
- 한국어 이해도 우수